In [1]:
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import nltk.corpus
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import datasets, linear_model
import scipy.sparse as sps
from scipy.sparse import coo_matrix, hstack, vstack,csr_matrix
from datetime import datetime
import gc
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer

## We have to load it everytime..Stupid..

In [2]:
# %load ../Library/LoggingForIPython.py
def printIpython(data,head=5,level=1):
    '''Level = 1,2,3,4 -> 4 is print everything'''
    currentLevel = 2
    if(level <= currentLevel ):
        print data.head(head),'\n'

# Logging Function

In [3]:
def myPrint(data,head=5,level=1):
    '''Level = 1,2,3,4 -> 4 is print everything'''
    currentLevel = 2
    if(level <= currentLevel ):
        print data.head(3),'\n'

## Test Log..Delete..

In [4]:
import logging
logger = logging.getLogger()
fhandler = logging.FileHandler(filename='mylog.log', mode='a')
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fhandler.setFormatter(formatter)
logger.addHandler(fhandler)
logger.setLevel(logging.DEBUG)

logging.error('hello!')
logging.debug('This is a debug message')
logging.info('this is an info message')
logging.warning('tbllalfhldfhd, warning.')

## Products..

In [5]:
#All columns can fit one line..
pd.set_option('expand_frame_repr', False)

products = pd.read_csv("../../input/producto_tabla.csv")
myPrint(products,3,2)
products['short_name'] = products.NombreProducto.str.extract('^(\D*)', expand=True)
myPrint(products,3,4)
products['brand'] = products.NombreProducto.str.extract('^.+\s(\D+) \d+$', expand=False)
myPrint(products,3,4)
w = products.NombreProducto.str.extract('(\d+)(Kg|g)', expand=True)
myPrint(products,3,4)
w.head(3)
products['weight'] = w[0].astype('float') * w[1].map({'Kg': 1000, 'g': 1})
products['pieces'] = products.NombreProducto.str.extract('(\d+)p ', expand=False).astype('float')
myPrint(products,3,4)
products['short_name_processed'] = (products['short_name'].map(
    lambda x: " ".join([i for i in x.lower().split() if i not in nltk.corpus.stopwords.words("spanish")])))
myPrint(products,3,4)
stemmer = SnowballStemmer("spanish")
products['short_name_processed'] = (
products['short_name_processed'].map(lambda x: " ".join([stemmer.stem(i) for i in x.lower().split()])))
myPrint(products,3,4)
products["pieces"].fillna(1, inplace=True)
products.drop(0,inplace=True)
products.drop(["short_name","NombreProducto"],axis=1,inplace=True)
products.head(5)
products.isnull().sum()
#products[pd.isnull(products).any(1)]
products[pd.isnull(products.brand)]
products[products.brand == 'BIM']
#Temporary solution..
products.fillna(products.mean(),inplace=True).head(5)
########################################
### Find missing values on internet!..
###Missing brand is bimbo!!!
######################################
#print products.shape
#print len(products.Producto_ID.unique())

   Producto_ID                          NombreProducto
0            0                       NO IDENTIFICADO 0
1            9               Capuccino Moka 750g NES 9
2           41  Bimbollos Ext sAjonjoli 6p 480g BIM 41 



,Producto_ID,brand,weight,pieces,short_name_processed
1,9,NES,750.0,1.0,capuccin mok
2,41,BIM,480.0,6.0,bimboll ext sajonjoli
3,53,LON,170.0,1.0,burrit sincr
4,72,TR,45.0,4.0,div tir mini doradit
5,73,BIM,540.0,1.0,pan multigran linaz


In [6]:
#train = pd.read_csv('../../input/train_100.csv')
#test = pd.read_csv('../../input/test_100.csv')
train = pd.read_csv('../../input/train.csv')
test = pd.read_csv('../../input/test.csv')


testIds = test['id']
test.drop(['id'],axis = 1,inplace=True)
trainY = train['Demanda_uni_equil']
trainX = train[test.columns.values]
print trainX.head(1)
print test.head(1)
trainTest = trainX.append(test,ignore_index=True)
print trainTest.head(1)
print "shape: " ,trainTest.shape
mergedTrainedTestProduct = trainTest.merge(products,on="Producto_ID",how="left")
mergedTrainedTestProduct.head(5)

   Semana  Agencia_ID  Canal_ID  Ruta_SAK  Cliente_ID  Producto_ID
0       3        1110         7      3301       15766         1212
   Semana  Agencia_ID  Canal_ID  Ruta_SAK  Cliente_ID  Producto_ID
0      11        4037         1      2209     4639078        35305
   Semana  Agencia_ID  Canal_ID  Ruta_SAK  Cliente_ID  Producto_ID
0       3        1110         7      3301       15766         1212
shape:  (81179715, 6)


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,brand,weight,pieces,short_name_processed
0,3,1110,7,3301,15766,1212,BIM,120.0,2.0,rol canel
1,3,1110,7,3301,15766,1216,BIM,135.0,2.0,rol glass
2,3,1110,7,3301,15766,1238,BIM,140.0,2.0,panquecit got choc
3,3,1110,7,3301,15766,1240,BIM,125.0,4.0,mantec vainill
4,3,1110,7,3301,15766,1242,BIM,105.0,6.0,donit espolvor


In [19]:
rowsize = test.shape[0]

del trainTest,test,trainX

gc.collect()

0

In [20]:
#brand names are single..
print products["brand"][products["brand"].astype(str).map(lambda x:len(x.split()))>1]

Series([], Name: brand, dtype: object)


In [21]:
#mergedTrainedTestProduct = mergedTrainedTestProduct[0:5]
mergedTrainedTestProduct = mergedTrainedTestProduct

#print mergedTrainedTestProduct

countvec = CountVectorizer()
shortname_sparse =  countvec.fit_transform(mergedTrainedTestProduct.short_name_processed)
#print shortname_sparse.toarray()
shortname_sparse



<81179715x542 sparse matrix of type '<type 'numpy.int64'>'
	with 145774286 stored elements in Compressed Sparse Row format>

# Brand Dummy Variable..

In [22]:
brand_sparse = countvec.fit_transform(mergedTrainedTestProduct.brand)
#print brand_sparse.toarray()
brand_sparse

<81179715x33 sparse matrix of type '<type 'numpy.int64'>'
	with 81179715 stored elements in Compressed Sparse Row format>

# Dropping unnecessary columns..

In [23]:
mergedTrainedTestProduct.drop(mergedTrainedTestProduct.columns[[0,1,2,3,4,5,6,9]],axis=1,inplace=True)
print mergedTrainedTestProduct.info()
mergedTrainedTestProduct.head(1)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 81179715 entries, 0 to 81179714
Data columns (total 2 columns):
weight    float64
pieces    float64
dtypes: float64(2)
memory usage: 1.8 GB
None


,weight,pieces
0,120.0,2.0


# Concatenating traintest data and sparse matrixes as sparse..

In [24]:
mergedTrainedTestProduct_sparse = coo_matrix(mergedTrainedTestProduct)
mergedTrainedTestProduct_sparse
#print fittedmergedTrainedTestProduct.astype(object).tocoo()
completeSparseData = hstack((mergedTrainedTestProduct_sparse, shortname_sparse,brand_sparse), format='csr')
#print completeSparseData.toarray()
completeSparseData

<81179715x577 sparse matrix of type '<type 'numpy.float64'>'
	with 389313431 stored elements in Compressed Sparse Row format>

In [28]:
del mergedTrainedTestProduct_sparse, shortname_sparse, brand_sparse
gc.collect()

NameError: name 'mergedTrainedTestProduct_sparse' is not defined

# if indexing doesn't work after merging, use below but looks like slow..
## But, we shouldn't drop semana column, it's working like if semana > 9, it is test..

In [13]:
#Use if necessary!!..
#print completeSparseData.toarray()
#print (completeSparseData[:,0]>9).toarray()
#indices = np.where((completeSparseData[:,0]>9).toarray())[0]
#indices
#completeSparseData[indices,:]

In [29]:
#print completeSparseData.toarray(), "\n"
train_sparse = completeSparseData[0:train.shape[0]]
test_sparse = completeSparseData[train.shape[0]:]
#print train_sparse.get_shape()
test_sparse

<6999251x577 sparse matrix of type '<type 'numpy.float64'>'
	with 33593313 stored elements in Compressed Sparse Row format>

In [30]:
del completeSparseData
gc.collect()

0

## Root mean square logarithmic square evaluation function..

In [31]:
def rmsle_eval(y, y0):
    y0=y0.get_label()    
    assert len(y) == len(y0)
    return 'error',np.sqrt(np.mean(np.power(np.log1p(y)-np.log1p(y0), 2)))

# Prediction..

In [ ]:
import xgboost as xgb

params = {}
params['objective'] = "reg:linear"
params['eta'] = 0.025
params['max_depth'] = 5
params['subsample'] = 0.8
params['colsample_bytree'] = 0.6
params['silent'] = True

test_preds = np.zeros(rowsize)

xg_train = xgb.DMatrix(train_sparse, label=trainY)
xg_test = xgb.DMatrix(test_sparse)

watchlist = [(xg_train, 'train')]
num_rounds = 100


xgclassifier = xgb.train(params, xg_train, num_rounds, watchlist, feval = rmsle_eval, 
                         early_stopping_rounds= 20, verbose_eval = 10)
#fold_preds = np.around(xgclassifier.predict(xg_test, ntree_limit=xgclassifier.best_iteration))
fold_preds = np.rint(xgclassifier.predict(xg_test, ntree_limit=xgclassifier.best_iteration))

test_preds += fold_preds

submission = pd.DataFrame({'id':testIds, 'Demanda_uni_equil': test_preds})
submission[["id","Demanda_uni_equil"]].to_csv('../../submissions/' + 
                                              datetime.now().strftime('%Y-%m-%d-%H-%M-%S') +'.csv', index=False)


In [74]:
submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 2 columns):
Demanda_uni_equil    100 non-null float64
id                   100 non-null int64
dtypes: float64(1), int64(1)
memory usage: 1.6 KB
